# ===============================================================
# 📒 NOTEBOOK 4: Model Optimization & Testing
# Hyperparameter tuning, advanced optimization, final testing
# ===============================================================


In [ ]:
# --- 1: Setup ---

!pip install tensorflow pandas numpy matplotlib scikit-learn kerastuner

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import pickle
import os
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print(f"✅ TensorFlow version: {tf.__version__}")

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# --- 2: Load Best Model ---

def load_best_model():
    """Load the best model from Notebook 3"""
    
    model = keras.models.load_model('/content/models/best_model.h5')
    
    # Load data
    X_test = np.load('/content/prepared_data/X_test.npy')
    y_test = np.load('/content/prepared_data/y_test.npy')
    
    with open('/content/label_encoder.pkl', 'rb') as f:
        label_encoder = pickle.load(f)
    
    print(f"✅ Model loaded")
    print(f"   Test data: {X_test.shape}")
    
    return model, X_test, y_test, label_encoder

model, X_test, y_test, label_encoder = load_best_model()

# One-hot encode
y_test_cat = tf.keras.utils.to_categorical(y_test, len(label_encoder.classes_))


In [ ]:
# --- 3: Hyperparameter Tuning (Optional) ---

import kerastuner as kt

def build_tuning_model(hp):
    """Build model for hyperparameter tuning"""
    
    model = keras.Sequential([
        keras.layers.Input(shape=X_test.shape[1:]),
        
        # Conv1D layers
        keras.layers.Conv1D(
            filters=hp.Int('conv1_filters', 32, 128, step=32),
            kernel_size=hp.Choice('conv1_kernel', [3, 5]),
            padding='same',
            activation='relu'
        ),
        keras.layers.BatchNormalization(),
        
        keras.layers.Conv1D(
            filters=hp.Int('conv2_filters', 64, 256, step=64),
            kernel_size=hp.Choice('conv2_kernel', [3, 5]),
            padding='same',
            activation='relu'
        ),
        keras.layers.MaxPooling1D(2),
        
        # LSTM or BiLSTM
        if hp.Boolean('use_bilstm'):
            keras.layers.Bidirectional(
                keras.layers.LSTM(
                    units=hp.Int('lstm_units', 64, 256, step=64),
                    return_sequences=False
                )
            )
        else:
            keras.layers.LSTM(
                units=hp.Int('lstm_units', 64, 256, step=64),
                return_sequences=False
            ),
        
        # Dense layers
        keras.layers.Dense(
            units=hp.Int('dense_units', 128, 512, step=128),
            activation='relu'
        ),
        keras.layers.Dropout(
            rate=hp.Float('dropout', 0.2, 0.5, step=0.1)
        ),
        
        keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Uncomment to run tuning (takes hours)
"""
tuner = kt.RandomSearch(
    build_tuning_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='/content/tuning',
    project_name='slsl_tuning'
)

tuner.search(
    X_train, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=30,
    batch_size=32,
    callbacks=[keras.callbacks.EarlyStopping(patience=5)]
)

best_hps = tuner.get_best_hyperparameters(5)
print(best_hps[0].values)
"""

print("✅ Hyperparameter tuning ready (uncomment to run)")


In [ ]:
# --- 4: Ensemble Prediction ---

def ensemble_predict(models, X):
    """Predict using ensemble of models"""
    
    predictions = []
    for model in models:
        pred = model.predict(X, verbose=0)
        predictions.append(pred)
    
    # Average predictions
    ensemble_pred = np.mean(predictions, axis=0)
    return ensemble_pred

# Load multiple models (if available)
models = [model]  # Add more models if you have them

if len(models) > 1:
    y_pred_ensemble = np.argmax(ensemble_predict(models, X_test), axis=1)
    y_true = np.argmax(y_test_cat, axis=1)
    
    ensemble_acc = np.mean(y_pred_ensemble == y_true)
    print(f"\n🎯 Ensemble Accuracy: {ensemble_acc:.4f}")


In [ ]:
# --- 5: Confidence Calibration ---

def calibrate_confidence(y_pred_probs, y_true):
    """Analyze model confidence"""
    
    import matplotlib.pyplot as plt
    
    y_pred = np.argmax(y_pred_probs, axis=1)
    confidences = np.max(y_pred_probs, axis=1)
    
    # Correct predictions
    correct_mask = (y_pred == y_true)
    correct_conf = confidences[correct_mask]
    wrong_conf = confidences[~correct_mask]
    
    plt.figure(figsize=(12, 4))
    
    # Histogram of confidences
    plt.subplot(1, 2, 1)
    plt.hist([correct_conf, wrong_conf], bins=20, 
             label=['Correct', 'Wrong'], alpha=0.7)
    plt.xlabel('Confidence')
    plt.ylabel('Count')
    plt.title('Confidence Distribution')
    plt.legend()
    
    # Reliability diagram
    plt.subplot(1, 2, 2)
    bins = np.linspace(0, 1, 11)
    correct_rates = []
    for i in range(len(bins)-1):
        mask = (confidences >= bins[i]) & (confidences < bins[i+1])
        if np.sum(mask) > 0:
            acc = np.mean(y_pred[mask] == y_true[mask])
            correct_rates.append(acc)
        else:
            correct_rates.append(0)
    
    plt.plot(bins[:-1] + 0.05, correct_rates, 'o-', label='Model')
    plt.plot([0, 1], [0, 1], '--', label='Perfect')
    plt.xlabel('Confidence')
    plt.ylabel('Accuracy')
    plt.title('Reliability Diagram')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('/content/calibration.png', dpi=150)
    plt.show()

y_pred_probs = model.predict(X_test)
calibrate_confidence(y_pred_probs, np.argmax(y_test_cat, axis=1))


In [ ]:
# --- 6: Test on New Videos ---

def test_on_new_video(video_path, model, label_encoder):
    """Test model on a completely new video"""
    
    # This would require the landmark extraction pipeline
    # You can implement this if you have new test videos
    
    print("🔄 To test new videos, first run them through Notebook 1")
    print("   Then load the landmarks and run inference")
    
    # Placeholder for implementation
    pass


In [ ]:
# --- 7: Final TFLite Optimization ---

def optimize_tflite_final(model):
    """Apply advanced TFLite optimizations"""
    
    # Try different quantization methods
    converters = {
        'float32': tf.lite.TFLiteConverter.from_keras_model(model),
        'float16': tf.lite.TFLiteConverter.from_keras_model(model),
        'int8': tf.lite.TFLiteConverter.from_keras_model(model)
    }
    
    # Float32 (baseline)
    converters['float32'].optimizations = []
    
    # Float16 quantization
    converters['float16'].optimizations = [tf.lite.Optimize.DEFAULT]
    converters['float16'].target_spec.supported_types = [tf.float16]
    
    # Int8 quantization (requires representative dataset)
    def representative_dataset():
        for i in range(100):
            yield [X_test[i:i+1].astype(np.float32)]
    
    converters['int8'].optimizations = [tf.lite.Optimize.DEFAULT]
    converters['int8'].representative_dataset = representative_dataset
    converters['int8'].target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converters['int8'].inference_input_type = tf.int8
    converters['int8'].inference_output_type = tf.int8
    
    results = {}
    for name, converter in converters.items():
        try:
            tflite_model = converter.convert()
            size = len(tflite_model) / 1024
            results[name] = {
                'model': tflite_model,
                'size_kb': size,
                'path': f'/content/models/model_{name}.tflite'
            }
            
            # Save
            with open(results[name]['path'], 'wb') as f:
                f.write(tflite_model)
            
            print(f"{name}: {size:.2f} KB")
            
        except Exception as e:
            print(f"{name}: Failed - {e}")
    
    return results

tflite_results = optimize_tflite_final(model)

# Compare sizes
print("\n📊 Model Size Comparison:")
for name, result in tflite_results.items():
    print(f"   {name}: {result['size_kb']:.2f} KB")


In [ ]:
# --- 8: Speed Benchmark ---

def benchmark_tflite(tflite_path, X_test, num_runs=100):
    """Benchmark TFLite inference speed"""
    
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Warmup
    for i in range(10):
        interpreter.set_tensor(input_details[0]['index'], X_test[i:i+1].astype(np.float32))
        interpreter.invoke()
    
    # Benchmark
    import time
    times = []
    
    for i in range(min(num_runs, len(X_test))):
        interpreter.set_tensor(input_details[0]['index'], X_test[i:i+1].astype(np.float32))
        
        start = time.time()
        interpreter.invoke()
        end = time.time()
        
        times.append((end - start) * 1000)  # ms
    
    avg_time = np.mean(times)
    std_time = np.std(times)
    
    print(f"\n⚡ Speed Benchmark ({num_runs} runs):")
    print(f"   Average: {avg_time:.2f} ms")
    print(f"   Std Dev: {std_time:.2f} ms")
    print(f"   Min: {np.min(times):.2f} ms")
    print(f"   Max: {np.max(times):.2f} ms")
    print(f"   FPS: {1000/avg_time:.1f} fps")
    
    return times

# Benchmark best model
best_model_path = '/content/models/sentence_model.tflite'
times = benchmark_tflite(best_model_path, X_test)

# Plot distribution
plt.figure(figsize=(10, 4))
plt.hist(times, bins=20, alpha=0.7)
plt.axvline(np.mean(times), color='red', linestyle='--', label=f'Mean: {np.mean(times):.2f}ms')
plt.xlabel('Inference Time (ms)')
plt.ylabel('Count')
plt.title('Inference Time Distribution')
plt.legend()
plt.savefig('/content/benchmark.png', dpi=150)
plt.show()


In [ ]:
# --- 9: Final Model Report ---

def generate_final_report(model, tflite_results, test_accuracy, times):
    """Generate comprehensive model report"""
    
    report = {
        'model_info': {
            'input_shape': str(model.input_shape),
            'output_shape': str(model.output_shape),
            'total_params': model.count_params(),
            'trainable_params': sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
        },
        'performance': {
            'test_accuracy': float(test_accuracy),
            'inference_time_ms': float(np.mean(times)),
            'inference_std_ms': float(np.std(times)),
            'fps': float(1000/np.mean(times))
        },
        'tflite_models': {
            name: {
                'size_kb': float(result['size_kb']),
                'path': result['path']
            } for name, result in tflite_results.items()
        }
    }
    
    # Save report
    with open('/content/final_model_report.json', 'w') as f:
        json.dump(report, f, indent=2)
    
    # Create text report
    with open('/content/final_model_report.txt', 'w') as f:
        f.write("="*60 + "\n")
        f.write("SLSL TRANSLATION MODEL - FINAL REPORT\n")
        f.write("="*60 + "\n\n")
        
        f.write("MODEL ARCHITECTURE:\n")
        f.write(f"  Input Shape: {report['model_info']['input_shape']}\n")
        f.write(f"  Total Parameters: {report['model_info']['total_params']:,}\n\n")
        
        f.write("PERFORMANCE:\n")
        f.write(f"  Test Accuracy: {report['performance']['test_accuracy']*100:.2f}%\n")
        f.write(f"  Inference Time: {report['performance']['inference_time_ms']:.2f} ms\n")
        f.write(f"  FPS: {report['performance']['fps']:.1f}\n\n")
        
        f.write("TFLITE MODELS:\n")
        for name, res in report['tflite_models'].items():
            f.write(f"  {name}: {res['size_kb']:.2f} KB\n")
    
    print("\n📄 Final report saved to /content/final_model_report.txt")
    return report

# Get test accuracy
test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
report = generate_final_report(model, tflite_results, test_acc, times)


In [ ]:
# --- 10: Prepare for Publication ---

def prepare_publication_package():
    """Prepare all files for publication/submission"""
    
    os.makedirs('/content/publication_package', exist_ok=True)
    
    # Copy all important files
    !cp /content/models/sentence_model.tflite /content/publication_package/
    !cp /content/final_model_report.txt /content/publication_package/
    !cp /content/training_history.png /content/publication_package/
    !cp /content/confusion_matrix.png /content/publication_package/
    !cp /content/per_class_accuracy.png /content/publication_package/
    !cp /content/calibration.png /content/publication_package/
    !cp /content/benchmark.png /content/publication_package/
    !cp /content/label_encoder.pkl /content/publication_package/
    
    # Create README
    with open('/content/publication_package/README.txt', 'w') as f:
        f.write("SLSL MEDICAL TRANSLATION MODEL\n")
        f.write("="*40 + "\n\n")
        f.write("Files included:\n")
        f.write("- sentence_model.tflite: Quantized model for mobile\n")
        f.write("- label_encoder.pkl: Label mapping\n")
        f.write("- final_model_report.txt: Detailed performance report\n")
        f.write("- *.png: Performance visualizations\n\n")
        f.write("To use in Flutter:\n")
        f.write("1. Copy sentence_model.tflite to assets/models/\n")
        f.write("2. Use label_encoder.pkl to map predictions to Sinhala text\n")
    
    # Zip everything
    !zip -r /content/slsl_publication_package.zip /content/publication_package/
    
    print("\n📦 Publication package ready: /content/slsl_publication_package.zip")

prepare_publication_package()


In [ ]:
# --- 11: Download Final Package ---

from google.colab import files

files.download('/content/slsl_publication_package.zip')

print("\n" + "="*60)
print("🎉 ALL NOTEBOOKS COMPLETE!")
print("="*60)
print("\n✅ Model ready for Flutter integration")
print("✅ Performance reports generated")
print("✅ TFLite optimized for mobile")
print("\n📱 Next: Use in Flutter app with tflite_flutter")